In [2]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df = pd.read_csv('../Datasets/cases/Human_Resources_Analytics/HR_comma_sep.csv')
X = df.drop('left', axis=1)
y = df['left']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25, stratify=y)


(10496, 18)

In [8]:
ohe_trf = make_column_transformer((OneHotEncoder(sparse_output=False, drop='first'), make_column_selector(dtype_include=object)),
                                  remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

X_train_trf = ohe_trf.fit_transform(X_train)
X_test_trf = ohe_trf.transform(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

cols = np.arange(1,18)
gammas = np.linspace(0.01,5, 20)
cs = np.linspace(0.01, 5, 20)
scores = []

for col in tqdm(cols):
    for c in cs:
        for g in gammas:
            pca = PCA(n_components=col).set_output(transform='pandas')
            X_train_pca = pca.fit_transform(X_train_trf)
            # X_train_pca = X_train_pca.iloc[ : , : col]  
            X_test_pca = pca.transform(X_test_trf)
            # X_test_pca = X_test_pca.iloc[ : , : col]

            svm = SVC(gamma=g, C=c, kernel='rbf')
            svm.fit(X_train_pca,y_train)
            y_pred = svm.predict(X_test_pca)
            scores.append([col,g,c,accuracy_score(y_test,y_pred), f1_score(y_test, y_pred, average='macro')])

df_scores = pd.DataFrame(scores, columns=['comp','gamma','C', 'accuracy', 'f1'])
df_scores.sort_values(by=['accuracy', 'f1'], ascending=False)

 12%|█▏        | 2/17 [1:06:32<8:10:26, 1961.78s/it]